In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D


ModuleNotFoundError: No module named 'tensorflow'

## Get the data from the data directory
We keep 80% data for training and 20% for validation

In [ ]:
data_dir = "data"

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123)


val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123)


## Create a map for the labels for the collected data

In [ ]:
class_indices = {class_label: idx for class_label, idx in enumerate(train_ds.class_names)}

## Set prefetch
We would like the data to be prefetched. Let's use AUTOTUNE for that.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

## Base model initialization

In [ ]:
vgg16_model = tf.keras.applications.VGG16(include_top=False)

## Create the model

In [ ]:
vgg16_model.trainable = False

# Adding our own custom head
# Start by taking the output feature maps from NASNETMobile
x = vgg16_model.output

# Convert to a single-dimensional vector by Global Average Pooling. 
# We could also use Flatten()(x) GAP is more effective reduces params and controls overfitting.
x = GlobalAveragePooling2D()(x)

# Adding a dense layer with 712 units
x = Dense(712, activation='relu')(x)

# Dropout 40% of the activations, helps reduces overfitting
x = Dropout(0.40)(x)

# The fianl layer will contain 4 output units (no of units = no of classes) with softmax function.
preds = Dense(7,activation='softmax')(x)

## Train the model

In [ ]:
# Construct the full model
model = Model(inputs=vgg16_model.input, outputs=preds)

model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(train_ds, epochs=5,validation_data=(val_ds))

## Testing the model

In [ ]:

import cv2

image = cv2.imread("data/smile/opencv_frame_53.jpg")

# Expand the dimensions of the image to 4
image_expand = tf.expand_dims(image, axis=0)

prediction = model.predict(image_expand)
print(prediction)
predicted_class_idx = tf.argmax(prediction[0])
predicted_class = class_indices[predicted_class_idx.numpy()]

print(f"Predicted class: {predicted_class}")